In [1]:
from google.cloud import secretmanager
def get_secret(secret_id, project_id):
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/latest"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("UTF-8")

project_id = "impressive-mile-162105"
s3_access_key_id = get_secret("noetl-demo-access-key-id", project_id)
s3_secret_access_key = get_secret("noetl-demo-secret-access-key", project_id)

In [10]:
import duckdb

duckdb.sql("INSTALL httpfs; LOAD httpfs;")
duckdb.sql("SET s3_endpoint='storage.googleapis.com';")
duckdb.sql(f"SET s3_access_key_id='{s3_access_key_id}';")
duckdb.sql(f"SET s3_secret_access_key='{s3_secret_access_key}';")

symbols_valid_meta_polars_df = duckdb.sql("""
    SELECT * FROM read_csv_auto('gs://noetl-samples/stock-market-dataset/symbols_valid_meta.csv');
""").pl()
print(symbols_valid_meta_polars_df.head())


shape: (5, 12)
┌────────┬────────┬──────────────────────┬──────────┬───┬───────────┬────────┬────────┬────────────┐
│ Nasdaq ┆ Symbol ┆ Security Name        ┆ Listing  ┆ … ┆ Financial ┆ CQS    ┆ NASDAQ ┆ NextShares │
│ Traded ┆ ---    ┆ ---                  ┆ Exchange ┆   ┆ Status    ┆ Symbol ┆ Symbol ┆ ---        │
│ ---    ┆ str    ┆ str                  ┆ ---      ┆   ┆ ---       ┆ ---    ┆ ---    ┆ str        │
│ str    ┆        ┆                      ┆ str      ┆   ┆ str       ┆ str    ┆ str    ┆            │
╞════════╪════════╪══════════════════════╪══════════╪═══╪═══════════╪════════╪════════╪════════════╡
│ Y      ┆ A      ┆ Agilent              ┆ N        ┆ … ┆ null      ┆ A      ┆ A      ┆ N          │
│        ┆        ┆ Technologies, Inc.   ┆          ┆   ┆           ┆        ┆        ┆            │
│        ┆        ┆ Com…                 ┆          ┆   ┆           ┆        ┆        ┆            │
│ Y      ┆ AA     ┆ Alcoa Corporation    ┆ N        ┆ … ┆ null      ┆ AA    

In [12]:
import duckdb

duckdb.sql("INSTALL httpfs; LOAD httpfs;")
duckdb.sql("SET s3_endpoint='storage.googleapis.com';")
duckdb.sql(f"SET s3_access_key_id='{s3_access_key_id}';")
duckdb.sql(f"SET s3_secret_access_key='{s3_secret_access_key}';")

df_polars = duckdb.sql("""
    SELECT * FROM read_parquet('s3://noetl-examples/kaggle/stock-market-dataset/*.parquet')
""").pl()
print(df_polars.head())

shape: (5, 12)
┌────────┬────────┬──────────────────────┬──────────┬───┬───────────┬────────┬────────┬────────────┐
│ Nasdaq ┆ Symbol ┆ Security Name        ┆ Listing  ┆ … ┆ Financial ┆ CQS    ┆ NASDAQ ┆ NextShares │
│ Traded ┆ ---    ┆ ---                  ┆ Exchange ┆   ┆ Status    ┆ Symbol ┆ Symbol ┆ ---        │
│ ---    ┆ str    ┆ str                  ┆ ---      ┆   ┆ ---       ┆ ---    ┆ ---    ┆ str        │
│ str    ┆        ┆                      ┆ str      ┆   ┆ str       ┆ str    ┆ str    ┆            │
╞════════╪════════╪══════════════════════╪══════════╪═══╪═══════════╪════════╪════════╪════════════╡
│ Y      ┆ A      ┆ Agilent              ┆ N        ┆ … ┆ null      ┆ A      ┆ A      ┆ N          │
│        ┆        ┆ Technologies, Inc.   ┆          ┆   ┆           ┆        ┆        ┆            │
│        ┆        ┆ Com…                 ┆          ┆   ┆           ┆        ┆        ┆            │
│ Y      ┆ AA     ┆ Alcoa Corporation    ┆ N        ┆ … ┆ null      ┆ AA    

In [13]:
from sqlalchemy import create_engine
pg_user = "noetl"
pg_password = "noetl"
pg_host = "db"
pg_port = "5434"
pg_db = "noetl"
engine = create_engine(f"postgresql+psycopg://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_db}")
df_pandas = df_polars.to_pandas()
table_name = "symbols_valid_meta"
df_pandas.to_sql(table_name, engine, if_exists='replace', index=False)

-1

In [14]:
import polars as pl

df_polars = pl.read_database_uri(
    "SELECT * FROM test_data_table",
    "postgresql://noetl:noetl@db:5434/noetl"
)
print(df_polars.head())

shape: (5, 7)
┌─────┬─────────┬──────┬─────────────────┬───────────┬──────────────────────┬──────────────────────┐
│ id  ┆ name    ┆ age  ┆ created_at      ┆ is_active ┆ meta_data            ┆ description          │
│ --- ┆ ---     ┆ ---  ┆ ---             ┆ ---       ┆ ---                  ┆ ---                  │
│ i32 ┆ str     ┆ i32  ┆ datetime[μs]    ┆ bool      ┆ str                  ┆ str                  │
╞═════╪═════════╪══════╪═════════════════╪═══════════╪══════════════════════╪══════════════════════╡
│ 1   ┆ Alice   ┆ 30   ┆ 2025-06-11      ┆ true      ┆ {"key_1":"value_1"," ┆ Line 1\nLine 2\nLine │
│     ┆         ┆      ┆ 08:08:39.481777 ┆           ┆ key_2":123…          ┆ 3                    │
│ 2   ┆ Bob     ┆ 25   ┆ 2025-06-11      ┆ true      ┆ {"key_1":"value_2"," ┆ This is a\nmultiline │
│     ┆         ┆      ┆ 08:08:39.481777 ┆           ┆ key_2":456…          ┆ descripti…           │
│ 3   ┆ Charlie ┆ 40   ┆ 2025-06-11      ┆ true      ┆ {"key_1":"value_3"," ┆